In [1]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import polars as pl

API_KEY= 'redacted'

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix


In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# First get a list of in-season sports
#  The sport 'key' from the response can be used to get odds in the next request

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

sports_response = requests.get(
    'https://api.the-odds-api.com/v4/sports', 
    params={
        'api_key': API_KEY
    }
)


if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

else:
    print('List of in season sports:', sports_response.json())
    
mylist = sports_response.json()

In [3]:
# Grab the 'sport' values from the json

def flatten_dict_values(d):
    flat_list = []
    
    if isinstance(d, dict):  # Check if it's a dictionary
        for value in d.values():
            if isinstance(value, (dict, list)):  # If value is a dict or list
                flat_list.extend(flatten_dict_values(value))  # Recursively flatten
            else:
                flat_list.append(value)  # Add the value to the list
    elif isinstance(d, list):  # If it's a list, iterate over its items
        for item in d:
            if isinstance(item, (dict, list)):  # If item is a dict or list
                flat_list.extend(flatten_dict_values(item))  # Recursively flatten
            else:
                flat_list.append(item)  # Add the item to the list
    
    return flat_list

flat_values = flatten_dict_values(mylist)
#print(flat_values)

every_sixth_value = flat_values[0::6]  # Start at index 5 (the 6th element) and take every 6th element
print(every_sixth_value)

['americanfootball_ncaaf', 'americanfootball_ncaaf_championship_winner', 'americanfootball_nfl', 'americanfootball_nfl_super_bowl_winner', 'aussierules_afl', 'baseball_mlb_world_series_winner', 'basketball_euroleague', 'basketball_nba', 'basketball_nba_championship_winner', 'basketball_nbl', 'basketball_ncaab', 'basketball_ncaab_championship_winner', 'basketball_wncaab', 'boxing_boxing', 'cricket_big_bash', 'cricket_odi', 'golf_masters_tournament_winner', 'golf_pga_championship_winner', 'golf_the_open_championship_winner', 'golf_us_open_winner', 'icehockey_nhl', 'icehockey_nhl_championship_winner', 'icehockey_sweden_allsvenskan', 'icehockey_sweden_hockey_league', 'mma_mixed_martial_arts', 'rugbyleague_nrl', 'soccer_australia_aleague', 'soccer_austria_bundesliga', 'soccer_belgium_first_div', 'soccer_conmebol_copa_libertadores', 'soccer_efl_champ', 'soccer_england_efl_cup', 'soccer_england_league1', 'soccer_england_league2', 'soccer_epl', 'soccer_fa_cup', 'soccer_fifa_world_cup_winner', 

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

#Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
#This will deduct from the usage quota
#The usage quota cost = [number of markets specified] x [number of regions specified]
#For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

SPORT = 'basketball_ncaab'

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

In [ ]:
# turn json file into df

df = pd.json_normalize(
    odds_json, 
    record_path=['bookmakers', 'markets', 'outcomes'], 
    meta=[
        'sport_title', 'commence_time', 'home_team', 'away_team',
        ['bookmakers', 'key'], ['bookmakers', 'title'], ['bookmakers', 'markets', 'key']
    ],
    errors='ignore'
)

# Display the first few rows
df.head()

In [ ]:
df.to_csv("my_dataframe.csv", index=False)

#df = pd.read_csv("my_dataframe.csv")

cols = ['name', 'home_team', 'away_team']
df[cols] = df[cols].apply(lambda col: col.apply(lambda x: ''.join(x.split())))
df['game'] = df['home_team'] + df['away_team']

df.head()

In [23]:
# Find the arbitrage

df2 = (
    df[df['price'] > 2] # Filter rows where price > 2.0
    .groupby(['game'])
    .filter(lambda x: x['name'].nunique() >= 2)  # Keep groups with 2 or more unique 'name' values
)

df2 = df2[['sport_title','name','game','price','commence_time','bookmakers.title']]

# Display the result
df2

,sport_title,name,game,price,commence_time,bookmakers.title
0,NCAAB,DePaulBlueDemons,SetonHallPiratesDePaulBlueDemons,18.00,2025-01-08T23:32:42Z,FanDuel
3,NCAAB,SetonHallPirates,SetonHallPiratesDePaulBlueDemons,8.00,2025-01-08T23:32:42Z,MyBookie.ag
4,NCAAB,DePaulBlueDemons,SetonHallPiratesDePaulBlueDemons,21.00,2025-01-08T23:32:42Z,DraftKings
6,NCAAB,DePaulBlueDemons,SetonHallPiratesDePaulBlueDemons,13.00,2025-01-08T23:32:42Z,BetMGM
8,NCAAB,DePaulBlueDemons,SetonHallPiratesDePaulBlueDemons,6.75,2025-01-08T23:32:42Z,BetRivers
92,NCAAB,BucknellBison,BucknellBisonLafayetteLeopards,4.10,2025-01-09T00:02:22Z,DraftKings
94,NCAAB,BucknellBison,BucknellBisonLafayetteLeopards,2.75,2025-01-09T00:02:22Z,MyBookie.ag
96,NCAAB,BucknellBison,BucknellBisonLafayetteLeopards,3.20,2025-01-09T00:02:22Z,FanDuel
98,NCAAB,BucknellBison,BucknellBisonLafayetteLeopards,2.20,2025-01-09T00:02:22Z,BetRivers
101,NCAAB,LafayetteLeopards,BucknellBisonLafayetteLeopards,2.05,2025-01-09T00:02:22Z,Bovada


In [ ]:
# Save to CSV
csv_file_path = 'final.csv'
df2.to_csv(csv_file_path, index=False)

In [17]:
# Send email notification to interested parties


import smtplib
import ssl
from email.message import EmailMessage

# Define email sender and receiver
email_sender = 'redacted'
email_password = 'redacted'
email_receivers = ['redacted', 'redacted']

# Set the subject and body of the email
subject = 'Time to make a bet!'
body = """
See attached CSV file.
"""

# Create the email message
em = EmailMessage()
em['From'] = email_sender
em['To'] = ', '.join(email_receivers)
em['Subject'] = subject
em.set_content(body)

# Attach the CSV file
csv_file_path = "final.csv"  # Replace with your actual file path

# Open the file in binary mode and attach it
with open(csv_file_path, 'rb') as file:
    em.add_attachment(
        file.read(), 
        maintype='application',
        subtype='octet-stream',
        filename='InfiniteMoney.csv'
    )

# Add SSL (layer of security)
context = ssl.create_default_context()

# Log in and send the email
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(email_sender, email_password)
    smtp.sendmail(email_sender, email_receivers, em.as_string())